In [1092]:
import numpy as np
import matplotlib.pyplot as plt
from tpg import TPG, sample_tpg, sample_tpg_2, sample_tpg_h16
from kinodynamic_feasible import velocity_profile, kinodynamic_feasible_aux
from Bernstein import BernsteinPolynomial
import matplotlib.collections as collections
from math import sqrt
import matplotlib

In [1093]:
from collections import defaultdict
import copy
import numpy as np
from scipy.optimize import least_squares
from scipy.linalg import lstsq
from atpg import uniform_agent_aug_tpg, get_agent
from tpg import TPG, sample_tpg, sample_tpg_2, sample_tpg_h16

class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
        self.hascycle = False

    def addEdge(self, u, v):
        self.graph[u].append(v)

    def addEdgesFromTPG(self, tpg_data):
        # Apply uniform_agent_aug_tpg to get augmented TPG
        atpg = uniform_agent_aug_tpg(tpg_data, dp=None, dp_safe=None)

        # Iterate over the augmented TPG and add directed edges
        for agent in atpg:
            for node in agent:
                node_name = node.name
                next_node = node.next_node

                if next_node:
                    next_node_name = next_node.name
                    self.addEdge(node_name, next_node_name)

                type2_edge = node.type2
                if type2_edge:
                    type2_edge_name = type2_edge.name
                    self.addEdge(node_name, type2_edge_name)

    def DFSUtil(self, v, visited, recursionStack):
        visited.add(v)
        recursionStack.add(v)
        print(v, end=" ")

        for neighbour in self.graph[v]:
            if neighbour not in visited:
                self.DFSUtil(neighbour, visited, recursionStack)
            elif neighbour in recursionStack:
                print("\n")
                print(f"Cycle detected involving node {neighbour}")
                return True

        recursionStack.remove(v)
        return False

    def DFSAll(self):
        visited = set()
        recursionStack = set()

        # Create a copy of the keys to avoid "RuntimeError: dictionary changed size during iteration"
        nodes = list(self.graph.keys())  # No need to deepcopy, just create a list

        has_cycle = False

        for node in nodes:
            if node not in visited:
                print("\n")
                print(f"DFS starting from node {node}:")
                if self.DFSUtil(node, visited, recursionStack):
                    has_cycle = True
                    break  # Exit the loop if a cycle is detected
        
        self.hascycle = has_cycle  # Store the value in the instance variable

        print("\n")
        if has_cycle:
            print("The TPG contains a cycle.")
        else:
            print("The TPG does NOT contain any cycles.")

    def getHasCycle(self):
        return self.hascycle

    def generate_control_points(self, tpg_data):
        control_points = []
        unique_agents = set()

        for agent_trajectory in tpg_data:
            # Extract trajectory data from the TPG
            trajectory_data = []
            for node in agent_trajectory:
                node_name, time_step = node.name.split('^')[0][2:], int(node.name.split('_')[1])
                trajectory_data.append((node_name, time_step))
                unique_agents.add(node_name)
                # print(trajectory_data)
                # print(time_step)

            # Sort trajectory data by time step
            trajectory_data.sort(key=lambda x: x[1])
            # print(trajectory_data)
            # print(len(trajectory_data))

            # Calculate time intervals
            time_intervals = [trajectory_data[i+1][1] - trajectory_data[i][1] for i in range(len(trajectory_data)-1)]
            # print(time_intervals)

            # Determine the number of control points based on the number of trajectory data points
            num_control_points = time_step + 1
            # print(num_control_points)

            # Adjust positions based on the number of points
            positions = np.linspace(0, num_control_points-1, num_control_points)
            # print(positions)

            # Create Vandermonde matrix
            A = np.vander(positions, num_control_points, increasing=True)
            # print(A)

            prev_tinterval = 0

            # Iterate over segments and calculate control points
            for i, (segment_data, time_interval) in enumerate(zip(trajectory_data[:-1], time_intervals)):
                # Filter out empty strings from segment_data
                segment_data = [data for data in segment_data if data != '']
                # print(segment_data)

                # Filter out empty strings from segment_trajectory
                segment_trajectory = [float(data) for data in segment_data if data]
                # print(segment_trajectory)

                # Reshape segment_trajectory to a column vector
                segment_trajectory = np.array(segment_trajectory, dtype=np.float64).reshape((-1, 1))
                # print(segment_trajectory)

                # Adjust the shape of A to match the segment trajectory
                A_segment = A[:len(segment_trajectory)]

                # Calculate segment control points using ordinary least squares
                segment_control_points = np.linalg.lstsq(A_segment, segment_trajectory, rcond=None)[0]

                # Print the time intervals and positions
                print("Time Intervals:", prev_tinterval + time_interval, '/', num_control_points)
                prev_tinterval = prev_tinterval + time_interval
                print("Positions:", segment_trajectory.flatten())

                # Print the nodes corresponding to each time step
                segment_nodes = [node for node in segment_data]
                print("Nodes:", segment_nodes)

                # Append control points to the overall list
                control_points.append({
                    "Control Points": segment_control_points.tolist(),
                })

        num_agents = len(unique_agents)
        return control_points[:num_agents]

In [1094]:
# Create an instance of the Graph class
g = Graph()

# Add edges to the graph from the TPG data
tpg_data = sample_tpg_2()  # Sample TPG data for demonstration
g.addEdgesFromTPG(tpg_data)

# Perform DFS and check if the TPG contains cycles
g.DFSAll()

B


DFS starting from node A^1_0:
A^1_0 post_A^1_0 pre_B^1_2 B^1_2 post_B^1_2 pre_C^1_3 C^1_3 

DFS starting from node D^2_0:
D^2_0 post_D^2_0 pre_B^2_1 B^2_1 post_B^2_1 pre_E^2_2 E^2_2 

The TPG does NOT contain any cycles.


In [1095]:
# bounds = ((0, 1), (-1, 1))
# start_state = [0.6, 1]
# end_state = [0.6, 0]

# L = 0.5
# T = np.linspace(0.1, 10, 100)

t = 1.00000000003
n_cp = 4
# print(n_cp)
# print(tpg_data)
# print(len(tpg_data))

In [1097]:
# Generate control points
print(tpg_data)
ctrlpts = g.generate_control_points(tpg_data)
for i, agent_ctrlpts in enumerate(ctrlpts):
    print("Agent", i+1, "Control Points:", agent_ctrlpts)
# for i, agent_ctrlpts in enumerate(ctrlpts):
#     print("Agent", i+1, "Control Points:", agent_ctrlpts)

#     # Create a new figure and subplot for the current agent
#     fig, (ax1, ax2) = plt.subplots(2, 1)

#     # Create the Bernstein polynomial for the current agent
#     polynomial = BernsteinPolynomial(agent_ctrlpts['control_points'], T=1)

#     # Plot the control points on ax1
#     polynomial.plot(show_control_points=True, color="C1", ax=ax1)
#     ax1.set_xlabel('Time')
#     ax1.set_ylabel('Control Points')
#     ax1.set_title('Agent {} Control Points'.format(i+1))

#     # Calculate the derivative of the Bernstein polynomial
#     velocity = polynomial.deriv()

#     # Generate points on the velocity profile curve
#     t_points, velocity_points = velocity.plot(show_control_points=False, ret_points=True, ax=ax2)

#     # Plot the velocity profile on ax2
#     ax2.plot(t_points, velocity_points)
#     ax2.set_xlabel('Time')
#     ax2.set_ylabel('Velocity')
#     ax2.set_title('Agent {} Velocity Profile'.format(i+1))

#     # Adjust the layout to avoid overlapping
#     plt.tight_layout()

#     # Show the plot
#     plt.show()


[A^1_0, B^1_2, C^1_3]
[D^2_0, B^2_1 (B^1_2), E^2_2]
Time Intervals: 2 / 4
Positions: []
Nodes: [0]
Time Intervals: 3 / 4
Positions: [2.]
Nodes: [2]
Time Intervals: 1 / 3
Positions: []
Nodes: [0]
Time Intervals: 2 / 3
Positions: [1.]
Nodes: [1]
Agent 1 Control Points: {'Control Points': [[0.0], [0.0], [0.0], [0.0]]}
